In [1]:
# !pip freeze | grep scikit-learn

In [2]:
# !python -V

In [3]:
import pickle
import pandas as pd
import os

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/viviensiew/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/viviensiew/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

## Homework

In this homework, we'll deploy the ride duration model in batch mode. Like in homework 1, we'll use the Yellow Taxi Trip Records dataset. 

You'll find the starter code in the [homework](homework) directory.

Solution: [homework_solution/](homework_solution/)


## Q1. Notebook

We'll start with the same notebook we ended up with in homework 1.
We cleaned it a little bit and kept only the scoring part. You can find the initial notebook [here](homework/starter.ipynb).

Run this notebook for the March 2023 data.

What's the standard deviation of the predicted duration for this dataset?

* 1.24
* 6.24
* 12.28
* 18.28

In [8]:
y_pred.std()

np.float64(6.247488852238703)

## Q2. Preparing the output

Like in the course videos, we want to prepare the dataframe with the output. 

First, let's create an artificial `ride_id` column:

```python
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
```

Next, write the ride id and the predictions to a dataframe with results. 

Save it as parquet:

```python
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
```

What's the size of the output file?

* 36M
* 46M
* 56M
* 66M

__Note:__ Make sure you use the snippet above for saving the file. It should contain only these two columns. For this question, don't change the
dtypes of the columns and use `pyarrow`, not `fastparquet`.

In [9]:
year = 2023
month = 3
output_file = 'output.parquet'

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['prediction'] = y_pred

# write the ride id and the predictions to a dataframe with results.
df_result = df[['ride_id', 'prediction']]
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3316216 entries, 0 to 3403765
Data columns (total 2 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ride_id     object 
 1   prediction  float64
dtypes: float64(1), object(1)
memory usage: 75.9+ MB


In [10]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
# get file size in bytes, then divide by 1024**2 to get size in MB
print(f'Output file size: {os.path.getsize(output_file) / (1024 ** 2) :.0f}MB')

Output file size: 65MB


## Q3. Creating the scoring script

Now let's turn the notebook into a script. 

Which command you need to execute for that?

**Answer**:

```bash
jupyter nbconvert --to script starter.ipynb
```